### Import Data:

In [7]:
import pandas as pd
ratings = pd.read_csv('../Datasets/kaggle_book_recommendation/Ratings.csv')
books = pd.read_csv('../Datasets/kaggle_book_recommendation/Books.csv', dtype={3: 'str'})

In [11]:
len(ratings)

1149780

In [4]:
ratings['User-ID'].nunique()

105283

In [7]:
ratings['ISBN'].nunique()

340556

In [10]:
books['ISBN'].nunique()

271360

In [14]:
books['Book-Title'].nunique()

242135

### Process the Data:

Merge ratings dataframe with the books dataframe and remove unnecessary columns:

In [8]:
ratings_books_df = ratings.merge(books, on='ISBN')
ratings_books_df.drop(columns=['ISBN', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
ratings_books_df.columns = ['user', 'rating', 'title']
ratings_books_df.head()

,user,rating,title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel


In [9]:
len(ratings_books_df)

1031136

Filter the Data:

In [10]:
# Remove all ratings with 0.
ratings_books_df = ratings_books_df[ratings_books_df['rating'] > 0]

users_with_5_ratings = ratings_books_df.groupby('user').filter(lambda x: len(x) == 5)

In [11]:
df_5 = users_with_5_ratings[users_with_5_ratings['title'].isin(ratings_books_df['title'])]

In [12]:
df_5 = df_5.sort_values(by='user')

In [13]:
df_5

,user,rating,title
188105,183,8,Folio Junior: L'histoire De Monsieur Sommer
188115,183,9,Fahrenheit 451
188121,183,7,Cuentos del Planeta Tierra
188126,183,9,Que Se Mueran Los Feos (Fabula)
188127,183,8,Estudios sobre el amor
...,...,...,...
180703,278668,5,El Alquimista
180709,278668,5,LA Muerte Del Decano
180716,278668,5,Nadie Conoce a Nadie (Tiempos modernos)
180710,278668,5,La Perla del Oriente (ColecciÃ³n Ancora y delf...


In [14]:
df_5['comb'] = '"' + df_5['title'] + '" ' + df_5['rating'].astype(str)

In [15]:
df_5

,user,rating,title,comb
188105,183,8,Folio Junior: L'histoire De Monsieur Sommer,"""Folio Junior: L'histoire De Monsieur Sommer"" 8"
188115,183,9,Fahrenheit 451,"""Fahrenheit 451"" 9"
188121,183,7,Cuentos del Planeta Tierra,"""Cuentos del Planeta Tierra"" 7"
188126,183,9,Que Se Mueran Los Feos (Fabula),"""Que Se Mueran Los Feos (Fabula)"" 9"
188127,183,8,Estudios sobre el amor,"""Estudios sobre el amor"" 8"
...,...,...,...,...
180703,278668,5,El Alquimista,"""El Alquimista"" 5"
180709,278668,5,LA Muerte Del Decano,"""LA Muerte Del Decano"" 5"
180716,278668,5,Nadie Conoce a Nadie (Tiempos modernos),"""Nadie Conoce a Nadie (Tiempos modernos)"" 5"
180710,278668,5,La Perla del Oriente (ColecciÃ³n Ancora y delf...,"""La Perla del Oriente (ColecciÃ³n Ancora y del..."


In [16]:
# Grouping by 'user' and joining 'comb' values
users_filtered = df_5.groupby('user')['comb'].agg('\n'.join).reset_index()

In [17]:
# Extracting the last sentence from each 'combined' entry
users_filtered['last'] = users_filtered['comb'].apply(lambda x: x.split('\n')[-1])

In [18]:
users_filtered['ans'] = users_filtered['last'].apply(lambda x: x.split(' ')[-1])

In [19]:
users_filtered['query'] = users_filtered['last'].apply(lambda x: ' '.join(x.split()[:-1]))

In [20]:
users_filtered['data'] = users_filtered['comb'].apply(lambda x: '\n'.join(x.split('\n')[:-1]))

In [21]:
users_filtered['prompt'] = users_filtered.apply(lambda row: f"""You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User {row['user']} has rated the following books:

{row['data']}

Predict the user's rating on {row['query']}. Output the rating score only. Do not include other text.""", axis=1)

In [22]:
users_filtered

,user,comb,last,ans,query,data,prompt
0,183,"""Folio Junior: L'histoire De Monsieur Sommer"" ...","""Estudios sobre el amor"" 8",8,"""Estudios sobre el amor""","""Folio Junior: L'histoire De Monsieur Sommer"" ...",You are a book recommender and your job is to ...
1,387,"""The Adventures of Tom Sawyer"" 9\n""Santa Broug...","""Jane Eyre"" 9",9,"""Jane Eyre""","""The Adventures of Tom Sawyer"" 9\n""Santa Broug...",You are a book recommender and your job is to ...
2,408,"""Alice's Tulips"" 8\n""Guinness World Records 20...","""Frankenstein (Illustrated Classics Series)"" 7",7,"""Frankenstein (Illustrated Classics Series)""","""Alice's Tulips"" 8\n""Guinness World Records 20...",You are a book recommender and your job is to ...
3,472,"""Organic Chemistry (Saunders Golden Sunburst S...","""Secrets of St. John's Wort: A Lynn Sonberg Bo...",5,"""Secrets of St. John's Wort: A Lynn Sonberg Book""","""Organic Chemistry (Saunders Golden Sunburst S...",You are a book recommender and your job is to ...
4,566,"""Who Moved My Cheese? An Amazing Way to Deal w...","""New Cardiff"" 6",6,"""New Cardiff""","""Who Moved My Cheese? An Amazing Way to Deal w...",You are a book recommender and your job is to ...
...,...,...,...,...,...,...,...
1998,278019,"""U.S.S. Seawolf"" 6\n""Ancient Blood: Day of Ho...","""Armageddon Sky (Star Trek Deep Space Nine: Da...",9,"""Armageddon Sky (Star Trek Deep Space Nine: Da...","""U.S.S. Seawolf"" 6\n""Ancient Blood: Day of Ho...",You are a book recommender and your job is to ...
1999,278565,"""Gilbert: A Comedy of Manners"" 8\n""Murder at t...","""The Heretic's Apprentice: The Sixteenth Chron...",8,"""The Heretic's Apprentice: The Sixteenth Chron...","""Gilbert: A Comedy of Manners"" 8\n""Murder at t...",You are a book recommender and your job is to ...
2000,278637,"""L'Envoi Des Anges"" 7\n""La Muraille invisible""...","""Je vous Ã?Â©cris"" 7",7,"""Je vous Ã?Â©cris""","""L'Envoi Des Anges"" 7\n""La Muraille invisible""...",You are a book recommender and your job is to ...
2001,278663,"""The Bean Trees"" 4\n""A Woman's Place"" 5\n""Even...","""Special Relationship"" 3",3,"""Special Relationship""","""The Bean Trees"" 4\n""A Woman's Place"" 5\n""Even...",You are a book recommender and your job is to ...


In [23]:
df_llm_5_items = users_filtered[['prompt', 'query', 'ans']]

In [24]:
df_llm_5_items

,prompt,query,ans
0,You are a book recommender and your job is to ...,"""Estudios sobre el amor""",8
1,You are a book recommender and your job is to ...,"""Jane Eyre""",9
2,You are a book recommender and your job is to ...,"""Frankenstein (Illustrated Classics Series)""",7
3,You are a book recommender and your job is to ...,"""Secrets of St. John's Wort: A Lynn Sonberg Book""",5
4,You are a book recommender and your job is to ...,"""New Cardiff""",6
...,...,...,...
1998,You are a book recommender and your job is to ...,"""Armageddon Sky (Star Trek Deep Space Nine: Da...",9
1999,You are a book recommender and your job is to ...,"""The Heretic's Apprentice: The Sixteenth Chron...",8
2000,You are a book recommender and your job is to ...,"""Je vous Ã?Â©cris""",7
2001,You are a book recommender and your job is to ...,"""Special Relationship""",3


# ChatGPT 3.5 w/o finetune

In [25]:
from openai import OpenAI
client = OpenAI(api_key = 'YOUR-API-KEY')

# response = client.chat.completions.create(
#   model="gpt-3.5-turbo", 
#   messages=[
#     {"role": "system", "content": "You are a book recommender."},
#     {"role": "user", "content": "Who won the world series in 2020?"},
#     {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
#     {"role": "user", "content": "Where was it played?"}
#   ]
# )

In [52]:
total, false, true = 0,0,0
for idx, row in df_llm_5_items.iterrows():
    try:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[
            {"role": "system", "content": "You are a book recommender."},
            {"role": "user", "content": row['prompt']},
        ]
        )
        pred = int(response.choices[0].message.content.strip("'"))
        ans = int(row['ans'])
        total += 1
        print(f'{total}th pred {pred} ans {ans}')
        df_llm_5_items.at[idx, 'pred-ChatGPT-3.5'] = pred
        if pred != ans:
            false += 1
            #print(f"PROMPT {row['prompt']}")
        else:
            true += 1
    except Exception as e:
        print(f"Skipping index {idx} prompt {row['prompt']} reponse {response.choices[0].message.content} due to error: {e}")
print(f'total {total} # of wrongs {false} acc {true/total*100.00}')    

1th pred 7 ans 8
2th pred 8 ans 9
3th pred 7 ans 7
4th pred 8 ans 5
5th pred 7 ans 6
6th pred 7 ans 10
7th pred 7 ans 8
8th pred 7 ans 7
9th pred 7 ans 5
10th pred 8 ans 8
11th pred 8 ans 7
12th pred 7 ans 6
13th pred 9 ans 8
14th pred 7 ans 5
15th pred 7 ans 7
16th pred 6 ans 5
17th pred 7 ans 5
18th pred 7 ans 9
19th pred 8 ans 7
20th pred 8 ans 9
21th pred 8 ans 8
22th pred 8 ans 7
23th pred 8 ans 10
24th pred 8 ans 8
25th pred 6 ans 9
26th pred 7 ans 7
27th pred 7 ans 8
28th pred 7 ans 9
29th pred 8 ans 8
30th pred 8 ans 8
31th pred 7 ans 6
32th pred 8 ans 7
33th pred 6 ans 8
34th pred 7 ans 8
35th pred 7 ans 6
36th pred 8 ans 5
37th pred 9 ans 8
38th pred 4 ans 5
39th pred 7 ans 9
40th pred 7 ans 5
41th pred 8 ans 10
42th pred 7 ans 10
43th pred 8 ans 7
44th pred 5 ans 5
45th pred 8 ans 6
46th pred 7 ans 8
47th pred 7 ans 8
48th pred 8 ans 8
49th pred 8 ans 7
50th pred 7 ans 8
51th pred 7 ans 8
52th pred 7 ans 7
53th pred 5 ans 8
54th pred 7 ans 7
55th pred 8 ans 4
Skipping index 

In [53]:
df_llm_5_items

,prompt,query,ans,pred-ChatGPT-3.5
0,You are a book recommender and your job is to ...,"""Estudios sobre el amor""",8,7.0
1,You are a book recommender and your job is to ...,"""Jane Eyre""",9,8.0
2,You are a book recommender and your job is to ...,"""Frankenstein (Illustrated Classics Series)""",7,7.0
3,You are a book recommender and your job is to ...,"""Secrets of St. John's Wort: A Lynn Sonberg Book""",5,8.0
4,You are a book recommender and your job is to ...,"""New Cardiff""",6,7.0
...,...,...,...,...
1998,You are a book recommender and your job is to ...,"""Armageddon Sky (Star Trek Deep Space Nine: Da...",9,8.0
1999,You are a book recommender and your job is to ...,"""The Heretic's Apprentice: The Sixteenth Chron...",8,8.0
2000,You are a book recommender and your job is to ...,"""Je vous Ã?Â©cris""",7,8.0
2001,You are a book recommender and your job is to ...,"""Special Relationship""",3,7.0


# llama 3 8B w/o finetune

In [55]:
import transformers
import torch
import os
os.environ['HF_HOME'] = "/scratch/sjung61/huggingface"
cache_dir = "/scratch/sjung61/huggingface"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model="meta-llama/Meta-Llama-3-8B-Instruct",
  model_kwargs={"torch_dtype": torch.bfloat16, "use_auth_token": "YOUR-API-KEY"},
  device="cuda",
)

2024-04-27 19:13:03.518689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 19:13:04.519123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sjung61/.conda/envs/clenv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/sjung61/.conda/envs/clenv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/sjung61/.conda/envs/clenv/li

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [68]:
total, false, true = 0,0,0
for idx, row in df_llm_5_items.iterrows():
    try:
        messages = [
            {"role": "system", "content": "You are a book recommender."},
            {"role": "user", "content": row['prompt']},
        ]

        prompt = pipeline.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )

        terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = pipeline(
            prompt,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        
        pred = int(outputs[0]["generated_text"][len(prompt):])
        ans = int(row['ans'])
        total += 1
        print(f'{total}th pred {pred} ans {ans}')
        df_llm_5_items.at[idx, 'pred-llama3-8B'] = pred
        if pred != ans:
            false += 1
            #print(f"PROMPT {row['prompt']}")
        else:
            true += 1
    except Exception as e:
        print(f"Skipping index {idx} prompt {row['prompt']} reponse {response.choices[0].message.content} due to error: {e}")
        
print(f'total {total} # of wrongs {false} acc {true/total*100.00}')    

/home/sjung61/.conda/envs/clenv/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/sjung61/.conda/envs/clenv/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping index 0 prompt You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 183 has rated the following books:

"Folio Junior: L'histoire De Monsieur Sommer" 8
"Fahrenheit 451" 9
"Cuentos del Planeta Tierra" 7
"Que Se Mueran Los Feos (Fabula)" 9

Predict the user's rating on "Estudios sobre el amor". Output the rating score only. Do not include other text. reponse 7 due to error: invalid literal for int() with base 10: '8.3'
1th pred 8 ans 9
2th pred 7 ans 7
3th pred 9 ans 5


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


4th pred 7 ans 6
Skipping index 5 prompt You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 567 has rated the following books:

"On the Road (Penguin 20th Century Classics)" 10
"Expecting Adam: A True Story of Birth, Rebirth, and Everyday Magic" 8
"The Future of Leadership: Today's Top Leadership Thinkers Speak to Tomorrow's Leaders" 8
"I And Thou" 9

Predict the user's rating on "The Prince (Dover Thrift Editions)". Output the rating score only. Do not include other text. reponse 7 due to error: invalid literal for int() with base 10: '7.5'
5th pred 8 ans 8


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

1712th pred 8 ans 10
1713th pred 8 ans 8
1714th pred 8 ans 9


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping index 1996 prompt You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 277509 has rated the following books:

"The Interpretation of Dreams (Oxford World's Classics)" 8
"El anatomista" 8
"The Piano Tuner : A Novel" 9
"Et Si Cetait Vrai" 10

Predict the user's rating on "A hora de SertÃ³rio (Finisterra)". Output the rating score only. Do not include other text. reponse 7 due to error: invalid literal for int() with base 10: '9.5'
1715th pred 8 ans 10
1716th pred 8 ans 9


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping index 1999 prompt You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 278565 has rated the following books:

"Gilbert: A Comedy of Manners" 8
"Murder at the National Cathedral (Capital Crime Mysteries)" 9
"Glitz" 7
"Di and I" 8

Predict the user's rating on "The Heretic's Apprentice: The Sixteenth Chronicle of Brother Cadfael". Output the rating score only. Do not include other text. reponse 7 due to error: invalid literal for int() with base 10: '8.2'
1717th pred 8 ans 7
1718th pred 8 ans 3
1719th pred 8 ans 5
total 1719 # of wrongs 1354 acc 21.23327515997673


In [69]:
df_llm_5_items

,prompt,query,ans,pred-ChatGPT-3.5,pred-llama3-8B
0,You are a book recommender and your job is to ...,"""Estudios sobre el amor""",8,7.0,NaN
1,You are a book recommender and your job is to ...,"""Jane Eyre""",9,8.0,8.0
2,You are a book recommender and your job is to ...,"""Frankenstein (Illustrated Classics Series)""",7,7.0,7.0
3,You are a book recommender and your job is to ...,"""Secrets of St. John's Wort: A Lynn Sonberg Book""",5,8.0,9.0
4,You are a book recommender and your job is to ...,"""New Cardiff""",6,7.0,7.0
...,...,...,...,...,...
1998,You are a book recommender and your job is to ...,"""Armageddon Sky (Star Trek Deep Space Nine: Da...",9,8.0,8.0
1999,You are a book recommender and your job is to ...,"""The Heretic's Apprentice: The Sixteenth Chron...",8,8.0,NaN
2000,You are a book recommender and your job is to ...,"""Je vous Ã?Â©cris""",7,8.0,8.0
2001,You are a book recommender and your job is to ...,"""Special Relationship""",3,7.0,8.0
